<a href="https://colab.research.google.com/github/shusank8/GPT2/blob/main/GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("GPT 2 FROM SCRATCH BY SHUSANKET BASYAL")

GPT 2 FROM SCRATCH BY SHUSANKET BASYAL


In [2]:
# starting by loading the weights just to get the understanding of it from hugging face

In [3]:
from transformers import GPT2LMHeadModel

In [4]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124 M parameter
sd_hf = model_hf.state_dict()

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
for k,v in sd_hf.items():
  print(k,v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [9]:
# COOL HOW MODEL LOOKS AT THE GIVEN PROMPT AND TRIES TO CONTIUE ITS PATTERN EVEN THOUGH IT IS A PRETRAINED MODEL THAT JUST TRIES TO PREDICT THE NEXT TOKEN, IF WE HAVE EXAMPLE OF QUESTION, ANSWER IN THE PROMPT MODEL UNDERSTANDS IT AND CONTINUES TO GENERATE IN THE PATTERN OF QUESTION AND ANSWER
from transformers import pipeline
generator = pipeline("text-generation", model='gpt2')
generator("Question: What is your name? Answer: Hello, my name is shusanket basyal. Question: How old are you? Answer: I am 23 years old. Question: Where are you? Answer:I am in United States. Question: What are you doing in United States? ", max_length = 300, num_return_sequences = 5)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Question: What is your name? Answer: Hello, my name is shusanket basyal. Question: How old are you? Answer: I am 23 years old. Question: Where are you? Answer:I am in United States. Question: What are you doing in United States? ****************************** Question: What are your jobs? Answer: I am doing some cooking in front of my office for a local grocery store as a kitchen person. Question: What are your hobbies? Answer: I am doing laundry for my wife. Question: What are you best at? Answer:I am best on the battlefield. Question: What are your hobbies? ********** Reply: Are you taking any courses in your field? ********* Answer: I am doing a lot of reading on my computer. ********* Question: How do you think people are doing in your field? Answer: They are very nice people. Question: Why? Answer: I believe in this way. Question: What are you thinking? Answer: I've been asking my wife for a new job and she is very nice. Question: What can you do for the commu

In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [10]:
class GPTConfig:
  block_size = 256
  vocab_size = 65
  n_layer = 6
  n_head = 6
  n_embd = 384

In [ ]:
class MLP(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.m = nn.Sequential(
    self.c_fc = nn.Linear(config.n_embd, 4*config.n_embd),
    self.gelu = nn.GELU(approximate = 'tanh'),
    self.c_proj = nn.Linear(4*config.n_embd, config.n_embd))

  def froward(self, x):
    return self.m(x)

In [ ]:
class Block(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.ln_1 = nn.LayerNorm(config.n_embd)
    self.attn = CausalSelfAttention(config)
    self.ln2 = nn.LayerNorm(config.n_embd)
    self.mlp = MLP(config)

  def forward(self, x):
    x = x+self.attn(self.ln1(x))
    x = x+ self.mlp(self.ln2(x))
    return x


In [ ]:
class GPT(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.config = config

    self.transformer = nn.ModuleDict(dict(
        wte = nn.Embeding(config.vocab_size, config.n_embd),
        wpe = nn.Embeding(config.vocab_size, config.n_embd),
        h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
        ln_f = nn.LayerNorm(config.n_embd)
    ))
    self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias = False)